In [1]:
import requests
import json
import psycopg2
from psycopg2 import OperationalError, Error
from urllib.parse import urlencode

from datetime import datetime

In [2]:
# Connect to PostgreSQL database
def connect_to_postgres():
    conn = psycopg2.connect(
        host="localhost",       
        database="lombardia_air_quality", 
        user="airdata_user",    
        password="user"
    )
    return conn

In [3]:
# CREATE TABLE
def create_table_station():
    """
    Crea la tabella 'station' nel database se non esiste, includendo PostGIS.
    Gestisce gli errori e fornisce messaggi utili in caso di problemi.
    """
    try:
        conn = connect_to_postgres()
        cur = conn.cursor()

        try:
            cur.execute("""
                DROP TABLE IF EXISTS station;
                CREATE EXTENSION IF NOT EXISTS postgis;

                CREATE TABLE IF NOT EXISTS station (
                    idsensore TEXT PRIMARY KEY,
                    nometiposensore TEXT,
                    unitamisura TEXT,
                    idstazione TEXT,
                    nomestazione TEXT,
                    quota TEXT,
                    provincia TEXT,
                    comune TEXT,
                    storico TEXT,
                    datastart TIMESTAMP,
                    datastop TIMESTAMP,
                    utm_nord TEXT,
                    utm_est TEXT,
                    lat NUMERIC,
                    lng NUMERIC,
                    locationtxt TEXT,
                    location geometry(POINT, 4326)
                );
            """)
            conn.commit()

        except Error as e:
            print("Errore durante la creazione della tabella o l'estensione PostGIS:")
            print(e)

        finally:
            cur.close()
            conn.close()

    except OperationalError as conn_err:
        print("Errore di connessione al database:")
        print(conn_err)


In [4]:
# Fetch data from Dati Lombardia API

def fetch_data_from_api(api_url):
    """
    Fetch data from the API with specified limit and order
    
    Parameters:
    - api_url: Base URL for the API
    """
    
    # Append parameters to URL
    full_url = api_url
    print(f"Requesting data from: {full_url}")
    
    response = requests.get(full_url)
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

In [5]:
def insert_station_data(data):
    
    conn = connect_to_postgres()
    cur = conn.cursor()
    insert_rows = []

    for r in data:
        try:
            #datastart = datetime.fromisoformat(r.get('datastart')) if r.get('datastart') else None
            #datastop = datetime.fromisoformat(r.get('datastop')) if r.get('datastop') else None

            insert_rows.append((
                r.get('idsensore'),
                r.get('nometiposensore'),
                r.get('unitamisura'),
                r.get('idstazione'),
                r.get('nomestazione'),
                r.get('quota'),
                r.get('provincia'),
                r.get('comune'),
                r.get('storico'),
                r.get('datastart'),
                r.get('datastop'),
                r.get('utm_nord'),
                r.get('utm_est'),
                r.get('lat'),
                r.get('lng'),
                str(r.get('location')),  # location come stringa
                r.get('lng'),
                r.get('lat')
            ))
        except Exception as e:
            print(f"Errore parsing record stazione: {e}")

    if insert_rows:

        cur.executemany("""
            INSERT INTO station (
                idsensore, nometiposensore, unitamisura, idstazione, nomestazione, quota,
                provincia, comune, storico, datastart, datastop, utm_nord, utm_est,
                lat, lng, locationtxt, location
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326))
            ON CONFLICT (idsensore) DO NOTHING;
        """, insert_rows)

        conn.commit()
    else:
        print("Nessun dato valido da inserire.")

    cur.close()

In [6]:
# FINAL FUNCTION TO LOAD DATA
def load_data_from_api(api_url, table_name):
    """
    Carica i dati da un'API e li inserisce in una tabella PostgreSQL.

    Args:
        api_url (str): URL dell'API da cui prelevare i dati.
        table_name (str): Nome della tabella di destinazione.
        limit (int): Numero massimo di record da recuperare.
        order_by (str): Campo per ordinare i risultati.
    """
    try:
        # Fetch data from API
        print(f"Fetching records from API...")
        raw_data = fetch_data_from_api(api_url)
        
        # Show sample data
        print(f"Received {len(raw_data)} records from API")
        if raw_data:
            print("Sample data item:")
            print(json.dumps(raw_data[0], indent=2))
        else:
            print("Warning: API returned no data.")
            return

        # Create table
        print(f"Creating table '{table_name}' if it doesn't exist...")
        if table_name == "station":
            create_table_station()
            print(f"Table {table_name} created!")
        else:
            print(f"Nessuna funzione di creazione definita per la tabella '{table_name}'.")
            return

        # Insert data 
        print("Inserting data into table...")
        insert_station_data(raw_data)
        print("Process completed successfully!")

    except Exception as e:
        print(f"Error: {str(e)}")



In [7]:
# RUN TO CREATE AND LOAD DATA

# API URL
api_url = "https://www.dati.lombardia.it/resource/ib47-atvt.json"

# Define the table name for your data
table_name = "station"

# Create the table and load data (limit=1000, order_by="Data DESC")
load_data_from_api(api_url, table_name)


Fetching records from API...
Requesting data from: https://www.dati.lombardia.it/resource/ib47-atvt.json


Received 984 records from API
Sample data item:
{
  "idsensore": "12691",
  "nometiposensore": "Arsenico",
  "unitamisura": "ng/m\u00b3",
  "idstazione": "560",
  "nomestazione": "Varese v.Copelli",
  "quota": "383",
  "provincia": "VA",
  "comune": "Varese",
  "storico": "N",
  "datastart": "2008-04-01T00:00:00.000",
  "utm_nord": "5073728",
  "utm_est": "486035",
  "lat": "45.81697450",
  "lng": "8.82024911",
  "location": {
    "type": "Point",
    "coordinates": [
      8.82024911,
      45.8169745
    ]
  },
  ":@computed_region_6hky_swhk": "1",
  ":@computed_region_ttgh_9sm5": "1",
  ":@computed_region_af5v_nc64": "3"
}
Creating table 'station' if it doesn't exist...
Table station created!
Inserting data into table...
Process completed successfully!
